In [1]:
!pip install fake_useragent

In [ ]:
!pip install peewee

In [7]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
from tqdm import tqdm
from fake_useragent import UserAgent
import json
from peewee import *
import datetime
import sys
import os

In [8]:
database = SqliteDatabase('2ch.db')

class BaseModel(Model):
    class Meta:
        database = database

class Board(BaseModel):
    key = CharField(primary_key=True)
        
class Icon(BaseModel):
    name = CharField()
    num = IntegerField(primary_key=True)
    board = ForeignKeyField(Board, backref='icons')
    
class Thread(BaseModel):
    num=IntegerField(primary_key=True)
    subject=CharField()
    board = ForeignKeyField(Board, backref='threads')
    
class Post(BaseModel):
    num=IntegerField(primary_key=True)
    name=CharField()
    comment=CharField()
    number=IntegerField()
    timestamp=TimestampField()
    thread=ForeignKeyField(Thread, backref='posts')

class File(BaseModel):
    fullname=CharField()
    md5=CharField(primary_key=True)
    path=CharField()
    post=ForeignKeyField(Post, backref='files')
    
class Like(BaseModel):
    post=ForeignKeyField(Post, backref='likes')
    likes=IntegerField()
    dislikes=IntegerField()
    date=TimestampField()
    
database.connect()
database.create_tables([Board, Icon, Thread, Post, File, Like])

In [9]:
def save_board(board):
    with database.atomic():
        try:
            b = Board.create(
                key=board['Board']
            )
        except:
            b = Board.select().where(Board.key == board['Board'])[0]
        for icon in board['icons']:
            try:
                Icon.create(
                    name=icon['name'],
                    num=icon['num'],
                    board=b
                )
            except:
                pass
        for thread in board['threads']:
            time.sleep(0.3)
            try:
                t = Thread.create(
                    num=thread['num'],
                    subject=thread['subject'],
                    board = b
                )
            except:
                t = Thread.select().where(Thread.num == thread['num'])[0]
            thread_json = requests.get(f'https://2ch.hk/po/res/{t.num}.json', headers={'User-Agent': UserAgent().chrome})
            if thread_json.status_code == 200:
                for post in tqdm(thread_json.json()['threads'][0]['posts']):
                    try:
                        p = Post.create(
                            num=post['num'],
                            name=post['name'],
                            comment=post['comment'],
                            number=post['number'],
                            timestamp=post['timestamp'],
                            thread = t
                        )
                    except:
                        p = Post.select().where(Post.num == post['num'])
                    Like.create(
                        post=p,
                        likes=post['likes'],
                        dislikes=post['dislikes']
                    )
                    for f in post['files']:
                        try:
                            File.create(
                                fullname=f['fullname'],
                                md5=f['md5'],
                                path=f['path'],
                                post = p
                            )
                        except:
                            pass                    
                
thread_catalog_url = 'https://2ch.hk/po/catalog.json'
while True:
    catalog_json = requests.get(thread_catalog_url, headers={'User-Agent': UserAgent().chrome})
    save_board(catalog_json.json())
    time.sleep(5 * 60)


100%|██████████| 61/61 [00:00<00:00, 1764.94it/s]


100%|██████████| 21/21 [00:00<00:00, 1046.91it/s]


100%|██████████| 16/16 [00:00<00:00, 1122.58it/s]


100%|██████████| 44/44 [00:00<00:00, 1240.48it/s]


100%|██████████| 11/11 [00:00<00:00, 978.29it/s]


...

100%|██████████| 15/15 [00:00<00:00, 941.26it/s]


100%|██████████| 2/2 [00:00<00:00, 931.14it/s]


100%|██████████| 18/18 [00:00<00:00, 1059.18it/s]


100%|██████████| 5/5 [00:00<00:00, 760.25it/s]


ChunkedEncodingError: ('Connection broken: IncompleteRead(5642 bytes read, 4598 more expected)', IncompleteRead(5642 bytes read, 4598 more expected))